In [75]:
import pyspark 
from pyspark import SparkFiles
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

from datetime import datetime
from dotenv import load_dotenv              # environment variables
import os

In [76]:
# load the environment variables
load_dotenv()

# assign environment variables
PASSWORD = os.getenv('MariaDB_Password')

In [77]:
spark = SparkSession.builder.appName("Credit Card App").getOrCreate()

External API

In [78]:
url = 'https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json'

Extract Function

In [79]:
# extract api - https://stackoverflow.com/questions/41820977/how-to-save-json-data-fetched-from-url-in-pyspark
def extract_api(url):
    spark.sparkContext.addFile(url)                                         # adds a json file to local disk from the api
    dataframe = spark.read.json(SparkFiles.get('loan_data.json'))           # get absolute path to the file and convert to dataframe
    return dataframe

Loading Function

In [81]:
# load/write data to MariaDB
def load_to_db(dataframe, db_name, table_name, user, password):
    dataframe.write.format("jdbc") \
                    .mode("append") \
                    .option("url", f"jdbc:mysql://localhost:3306/{db_name}") \
                    .option("dbtable", table_name) \
                    .option("user", user) \
                    .option("password", password) \
                    .save()

In [83]:
loan_df = extract_api(url)

load_to_db(loan_df,                         # dataframe
           'creditcard_capstone',           # db_name
           'CDW_SAPP_LOAN_APPLICATION',     # table_name
           'root',                          # user_name
           PASSWORD)                        # password